In [12]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars "/usr/local/spark/jars/mongo-spark-connector_2.12-3.0.2.jar,/usr/local/spark/jars/mongo-java-driver-3.12.9.jar" pyspark-shell'

In [13]:
from pyspark.sql import SparkSession
# fix read bug, basically turn off sampling
spark = SparkSession.builder.appName("TwitterMongo") \
.config("spark.mongodb.input.database", "mongodb://localhost:27017/twitter") \
.config("spark.mongodb.input.uri", "mongodb://localhost:27017/twitter.tweets") \
.config("spark.mongodb.read.sql.inferSchema.mapTypes.enabled", "FALSE") \
.config("spark.mongodb.output.uri","mongodb://localhost:27017/twitter.tweets").getOrCreate()

In [14]:
#df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [15]:
# create a spark session
spark = SparkSession \
.builder \
.master("local") \
.appName("ABC") \
.config("spark.driver.memory", "15g") \
.config("spark.mongodb.read.connection.uri", "mongodb://localhost:27017/twitter") \
.config("spark.mongodb.write.connection.uri", "mongodb://localhost:27017/twitter") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:2.12-3.0.2') \
.getOrCreate()
# read data from mongodb collection "questions" into a dataframe "df"
df = spark.read \
.format("com.mongodb.spark.sql.DefaultSource") \
.option("uri", "mongodb://localhost:27017/twitter") \
.option("database", "twitter") \
.option("collection", "tweets") \
.load()
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- contributors: void (nullable = true)
 |-- coordinates: void (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |    |-- title: string (

In [16]:
df.show()

2023-05-11 17:07:48,152 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1307.2 KiB


+--------------------+------------+-----------+--------------------+------------------+--------------------+--------------------+--------------+--------------+---------+------------+----+-------------------+-------------------+-----------------------+---------------------+-------------------------+-------------------+-----------------------+---------------+----+-----+------------------+-----------+--------------------+-------------------+--------------------+-----------------------+-----------+-------------+---------+--------------------+--------------------+--------------------+-------------+---------+--------------------+
|                 _id|contributors|coordinates|          created_at|display_text_range|            entities|   extended_entities|extended_tweet|favorite_count|favorited|filter_level| geo|                 id|             id_str|in_reply_to_screen_name|in_reply_to_status_id|in_reply_to_status_id_str|in_reply_to_user_id|in_reply_to_user_id_str|is_quote_status|lang|plac

In [17]:
df.createOrReplaceTempView("tweets")

In [18]:
dfSQL = spark.sql("SELECT * FROM tweets WHERE lang = 'en'")
dfSQL.show()

2023-05-11 17:07:57,943 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1309.9 KiB


+--------------------+------------+-----------+--------------------+------------------+--------------------+-----------------+--------------------+--------------+---------+------------+----+-------------------+-------------------+-----------------------+---------------------+-------------------------+-------------------+-----------------------+---------------+----+--------------------+------------------+-----------+--------------------+-------------------+--------------------+-----------------------+-----------+-------------+---------+--------------------+--------------------+--------------------+-------------+---------+--------------------+
|                 _id|contributors|coordinates|          created_at|display_text_range|            entities|extended_entities|      extended_tweet|favorite_count|favorited|filter_level| geo|                 id|             id_str|in_reply_to_screen_name|in_reply_to_status_id|in_reply_to_status_id_str|in_reply_to_user_id|in_reply_to_user_id_str|is_q

In [19]:
dfSQL.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- contributors: void (nullable = true)
 |-- coordinates: void (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |    |-- title: string (

In [20]:
# select tweet id, geo, lang, quoted_status,quoted_status.geo 

dfOne = spark.sql("SELECT DISTINCT id,  text, quote_count, reply_count, retweet_count, favorite_count, geo, place, lang, \
                  quoted_status,quoted_status.geo, quoted_status.text, user.name, user.location   \
                  FROM tweets WHERE lang = 'en' AND text LIKE '%euro%'")

In [21]:
dfOne.show()

+-------------------+--------------------+-----------+-----------+-------------+--------------+----+-----+----+--------------------+----+--------------------+--------------------+-------------------+
|                 id|                text|quote_count|reply_count|retweet_count|favorite_count| geo|place|lang|       quoted_status| geo|                text|                name|           location|
+-------------------+--------------------+-----------+-----------+-------------+--------------+----+-----+----+--------------------+----+--------------------+--------------------+-------------------+
|1444780630374588417|RT @StockholmCF: ...|          0|          0|            0|             0|null| null|  en|                null|null|                null|Sir Boddington MB...|        Switzerland|
|1424665478170136578|@BillyHynes @Scot...|          0|          0|            0|             0|null| null|  en|                null|null|                null|Paul Harrison-Davies|               null|


In [23]:
df = spark.sql("SELECT DISTINCT text FROM tweets WHERE lang = 'en' AND text LIKE '%euro%'")


In [53]:
spark.sql("SELECT DISTINCT substring(created_at, 27, 4) as Year, \
          substring(created_at, 5, 3) as Month, \
          substring(created_at, 9, 2) as Day, \
          count(id) as TweetCount \
          FROM tweets \
          WHERE lang = 'en' AND text LIKE '%euro%' GROUP BY substring(created_at, 27, 4), \
          substring(created_at, 5, 3), \
          substring(created_at, 9, 2)").show()


+----+-----+---+----------+
|Year|Month|Day|TweetCount|
+----+-----+---+----------+
|2021|  Aug| 10|         5|
|2021|  Oct| 03|        10|
|2021|  Aug| 09|        20|
+----+-----+---+----------+



In [27]:
spark.sql("SELECT created_at, count(*) as count FROM tweets WHERE lang = 'en' AND text LIKE '%euro%' GROUP BY created_at ORDER BY created_at").show()


+--------------------+-----+
|          created_at|count|
+--------------------+-----+
|Mon Aug 09 07:21:...|    1|
|Mon Aug 09 09:10:...|    1|
|Mon Aug 09 09:34:...|    1|
|Mon Aug 09 12:38:...|    1|
|Mon Aug 09 14:59:...|    1|
|Mon Aug 09 15:51:...|    1|
|Mon Aug 09 16:32:...|    1|
|Mon Aug 09 16:37:...|    1|
|Mon Aug 09 17:06:...|    1|
|Mon Aug 09 17:24:...|    1|
|Mon Aug 09 18:20:...|    1|
|Mon Aug 09 18:48:...|    1|
|Mon Aug 09 18:51:...|    1|
|Mon Aug 09 19:28:...|    1|
|Mon Aug 09 19:34:...|    1|
|Mon Aug 09 19:43:...|    1|
|Mon Aug 09 20:36:...|    1|
|Mon Aug 09 21:06:...|    1|
|Mon Aug 09 21:50:...|    1|
|Mon Aug 09 23:53:...|    1|
+--------------------+-----+
only showing top 20 rows

